# Monografía - Matías Macías Gómez
### Parte 1 - Diciembre 2020
##### Especialización en Analítica y Ciencia de Datos
##### Universidad de Antioquia

In [ ]:
!pip install --upgrade pip
!pip install pymap3d==2.1.0
!pip install -U l5kit

**l5kit** es una libreria creada y mantenida por **lyft** la cual contiene toda las funciones necesarias para hacer una primera iteracion del modelo.

In [ ]:
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import scipy as sp
import gc
import os
import random
import sys

import matplotlib.pyplot as plt
import seaborn as sns

from IPython.core.display import display, HTML

# --- plotly ---
from plotly import tools, subplots
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
import plotly.io as pio
pio.templates.default = "plotly_dark"

# --- models ---
from sklearn import preprocessing
from sklearn.model_selection import KFold
import lightgbm as lgb
import xgboost as xgb
import catboost as cb

from typing import Dict

from tempfile import gettempdir
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision.models.resnet import resnet50
from tqdm import tqdm

from l5kit.configs import load_config_data
from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.dataset import AgentDataset, EgoDataset
#from l5kit.rasterization import build_rasterizer
from l5kit.rasterization.rasterizer_builder import build_rasterizer
from l5kit.evaluation import write_pred_csv, compute_metrics_csv, read_gt_csv, create_chopped_dataset
from l5kit.evaluation.chop_dataset import MIN_FUTURE_STEPS
from l5kit.evaluation.metrics import neg_multi_log_likelihood, time_displace
from l5kit.geometry import transform_points
from l5kit.visualization import PREDICTED_POINTS_COLOR, TARGET_POINTS_COLOR, draw_trajectory
from prettytable import PrettyTable
from pathlib import Path

Se debe crear una carpeta extra en el sistema, ya que la función **"create_chopped_dataset"**, asume que se esta corriendo el notebook de manera local e intenta escribir en el sistema de archivos de la carpeta de los datos en la cual no tiene permisos de escritura por lo cual se debe que sobre escribir la funcion y darle una nueva ubicacion para escribir el archivo CSV que tiene que crea para que funcione de manera correcta este notebook en el kernel de kaggle

In [ ]:
!mkdir -p scenes

## Configuración para el manejo de los datos


- Se le da al sistema el path de la base de datos.
- Se crea el objeto **dm --> Data Manager** el cual se encarga del manejo de los datos en formato **.zarr** *(compresion, descompresion, carga en memoria)*.
- Se carga el objeto de configuración provisto por lyft, el cual tiene los path para la carga de los diferentes datasets, y los atributos/parametros por defecto de las diferentes funciones contenidas en l5kit. Como es un archivo en formato **.yaml** python lo carga como un diccionario.

In [ ]:
# set env variable for data
os.environ["L5KIT_DATA_FOLDER"] = "/kaggle/input/lyft-motion-prediction-autonomous-vehicles"
dm = LocalDataManager(None)
# get config
cfg = load_config_data("../input/agent-motion-config/agent_motion_config.yaml")
cfg

## Construcción del modelo:
el modelo esta basado en una resnet pre-entrenada con la base de datos imageNet incluida en pytorch, a la cual se le configura entrada para recibir los datos de las imagenes generadas por el rasterizador y la ultima capa para que la salida este ajustada al problema.

- **Entrada** --> numero de frames históricos (Frames previos al punto de corte, para este caso 100) * 2 + 3 canales correspondientes al RGB de una imagen
- **Salida** --> (X, Y) de las coordenadas del agente * número de estados futuros

In [ ]:
def build_model(cfg: Dict) -> torch.nn.Module:
    # load pre-trained Conv2D model
    model = resnet50(pretrained=True)

    # change input channels number to match the rasterizer's output
    num_history_channels = (cfg["model_params"]["history_num_frames"] + 1) * 2
    num_in_channels = 3 + num_history_channels
    model.conv1 = nn.Conv2d(
        num_in_channels,
        model.conv1.out_channels,
        kernel_size=model.conv1.kernel_size,
        stride=model.conv1.stride,
        padding=model.conv1.padding,
        bias=False,
    )
    # change output size to (X, Y) * number of future states
    num_targets = 2 * cfg["model_params"]["future_num_frames"]
    model.fc = nn.Linear(in_features=2048, out_features=num_targets)

    return model

### Función de entrenamiento del modelo


Entrenamiento del modelo al cual se le entregan las imagenes rasterizadas de los datos de entrenamiento 

In [ ]:
def forward(data, model, device, criterion):
    inputs = data["image"].to(device)
    target_availabilities = data["target_availabilities"].unsqueeze(-1).to(device)
    targets = data["target_positions"].to(device)
    # Forward pass
    outputs = model(inputs).reshape(targets.shape)
    loss = criterion(outputs, targets)
    # not all the output steps are valid, but we can filter them out from the loss using availabilities
    loss = loss * target_availabilities
    loss = loss.mean()
    return loss, outputs

## Inicialización y formato de los datos

In [ ]:
from l5kit.data import DataManager
from l5kit.rasterization import *

def xbuild_rasterizer(cfg: dict, data_manager: DataManager) -> Rasterizer:
    """Factory function for rasterizers, reads the config, loads required data and initializes the correct rasterizer.
    Args:
        cfg (dict): Config.
        data_manager (DataManager): Datamanager that is used to require files to be present.
    Raises:
        NotImplementedError: Thrown when the ``map_type`` read from the config doesn't have an associated rasterizer
        type in this factory function. If you have custom rasterizers, you can wrap this function in your own factory
        function and catch this error.
    Returns:
        Rasterizer: Rasterizer initialized given the supplied config.
    """
    raster_cfg = cfg["raster_params"]
    map_type = raster_cfg["map_type"]
    dataset_meta_key = raster_cfg["dataset_meta_key"]

    render_context = RenderContext(
        raster_size_px=np.array(raster_cfg["raster_size"]),
        pixel_size_m=np.array(raster_cfg["pixel_size"]),
        center_in_raster_ratio=np.array(raster_cfg["ego_center"]),
        set_origin_to_bottom=raster_cfg["set_origin_to_bottom"],
    )

    filter_agents_threshold = raster_cfg["filter_agents_threshold"]
    history_num_frames = cfg["model_params"]["history_num_frames"]

    if map_type in ["py_satellite", "satellite_debug"]:
        sat_image = _load_satellite_map(raster_cfg["satellite_map_key"], data_manager)

        try:
            dataset_meta = _load_metadata(dataset_meta_key, data_manager)
            world_to_ecef = np.array(dataset_meta["world_to_ecef"], dtype=np.float64)
            ecef_to_aerial = np.array(dataset_meta["ecef_to_aerial"], dtype=np.float64)

        except (KeyError, FileNotFoundError):  # TODO remove when new dataset version is available
            world_to_ecef = get_hardcoded_world_to_ecef()
            ecef_to_aerial = get_hardcoded_ecef_to_aerial()

        world_to_aerial = np.matmul(ecef_to_aerial, world_to_ecef)
        if map_type == "py_satellite":
            return SatBoxRasterizer(
                render_context, filter_agents_threshold, history_num_frames, sat_image, world_to_aerial,
            )
        else:
            return SatelliteRasterizer(render_context, sat_image, world_to_aerial)

    elif map_type in ["py_semantic", "semantic_debug"]:
        semantic_map_filepath = data_manager.require(raster_cfg["semantic_map_key"])
        try:
            dataset_meta = _load_metadata(dataset_meta_key, data_manager)
            world_to_ecef = np.array(dataset_meta["world_to_ecef"], dtype=np.float64)
        except (KeyError, FileNotFoundError):  # TODO remove when new dataset version is available
            world_to_ecef = get_hardcoded_world_to_ecef()
        if map_type == "py_semantic":
            return SemBoxRasterizer(
                render_context, filter_agents_threshold, history_num_frames, semantic_map_filepath, world_to_ecef,
            )
        else:
            return SemanticRasterizer(render_context, semantic_map_filepath, world_to_ecef)

    elif map_type == "box_debug":
        return BoxRasterizer(render_context, filter_agents_threshold, history_num_frames)
    elif map_type == "stub_debug":
        return StubRasterizer(render_context)
    else:
        raise NotImplementedError(f"Rasterizer for map type {map_type} is not supported.")

In [ ]:
# ===== INIT DATASET
# Carga de los datos de entrenamiento  
train_cfg = cfg["train_data_loader"]
# Construcción de la imagen con el rasterizador incluido en la libreria de l5kit
rasterizer = build_rasterizer(cfg, dm)
# Carga y descompresion de los valores de los datos de entreamiento
train_zarr = ChunkedDataset(dm.require(train_cfg["key"])).open()
# Carga de los datos de los agentes
train_dataset = AgentDataset(cfg, train_zarr, rasterizer)
# Carga y formato final de los datos de entrenamiento, se hace una estrategia de K-fold, implementada en la libreria de l5kit
train_dataloader = DataLoader(train_dataset, shuffle=train_cfg["shuffle"], batch_size=train_cfg["batch_size"], 
                             num_workers=train_cfg["num_workers"])
print(train_dataset)

### TR --> Trafic Lights para esta primer iteración no se utilizaron estos datos

## Inicialización del modelo

In [ ]:
# ==== INIT MODEL
# Cargar el dispositivo, en caso de tener GPU con nucles CUDA disponibles utilizar ese hardware, en caso contrario utilizar la CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Pasarle el modelo configurado con los parametros del configurador al dispositivo
model = build_model(cfg).to(device)
# Cargar un optimizador
optimizer = optim.Adam(model.parameters(), lr=1e-3)
# Cargar y definir un criterio para el calculo de la perdida, en este caso "mean square error" (Error cuadrático medio)
criterion = nn.MSELoss(reduction="none")

## Entrenamiento del modelo

In [ ]:
# ==== TRAIN LOOP
tr_it = iter(train_dataloader)
progress_bar = tqdm(range(cfg["train_params"]["max_num_steps"]))
losses_train = []
for _ in progress_bar:
    try:
        data = next(tr_it)
    except StopIteration:
        tr_it = iter(train_dataloader)
        data = next(tr_it)
    model.train()
    torch.set_grad_enabled(True)
    loss, _ = forward(data, model, device, criterion)

    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    losses_train.append(loss.item())
    progress_bar.set_description(f"loss: {loss.item()} loss(avg): {np.mean(losses_train)}")

In [ ]:
print(data.keys())

### Gráfica de la perdida con el set de entrenamiento.

In [ ]:
plt.plot(np.arange(len(losses_train)), losses_train, label="train loss")
plt.legend()
plt.show()

## Ajuste de la función *create_chopped_data* para su correcto funcionamiento en el kernel de Kaggle

In [ ]:
import argparse
from pathlib import Path

import numpy as np
from zarr import convenience

from l5kit.data import ChunkedDataset, get_agents_slice_from_frames
from l5kit.data.zarr_utils import zarr_scenes_chop
from l5kit.dataset.select_agents import TH_DISTANCE_AV, TH_EXTENT_RATIO, TH_YAW_DEGREE, select_agents

from l5kit.evaluation.extract_ground_truth import *

MIN_FUTURE_STEPS = 10

def xcreate_chopped_dataset(
    zarr_path: str, th_agent_prob: float, num_frames_to_copy: int, num_frames_gt: int, min_frame_future: int
) -> str:
    """
    Create a chopped version of the zarr that can be used as a test set.
    This function was used to generate the test set for the competition so that the future GT(Ground Truth) is not in the data.
    Store:
     - a dataset where each scene has been chopped at `num_frames_to_copy` frames;
     - a mask for agents for those final frames based on the original mask and a threshold on the future_frames;
     - the GT csv for those agents
     For the competition, only the first two (dataset and mask) will be available in the notebooks
    Args:
        zarr_path (str): input zarr path to be chopped
        th_agent_prob (float): threshold over agents probabilities used in select_agents function
        num_frames_to_copy (int):  number of frames to copy from the beginning of each scene, others will be discarded
        min_frame_future (int): minimum number of frames that must be available in the future for an agent
        num_frames_gt (int): number of future predictions to store in the GT file
    Returns:
        str: the parent folder of the new datam
    """
    zarr_path = Path(zarr_path)
    dest_path = Path("scenes") / f"{zarr_path.stem}_chopped_{num_frames_to_copy}"
    chopped_path = dest_path / zarr_path.name
    gt_path = dest_path / "gt.csv"
    mask_chopped_path = dest_path / "mask"

    # Create standard mask for the dataset so we can use it to filter out unreliable agents
    zarr_dt = ChunkedDataset(str(zarr_path))
    zarr_dt.open()

    agents_mask_path = Path(zarr_path) / f"agents_mask/{th_agent_prob}"
    if not agents_mask_path.exists():  # don't check in root but check for the path
        select_agents(
            zarr_dt,
            th_agent_prob=th_agent_prob,
            th_yaw_degree=TH_YAW_DEGREE,
            th_extent_ratio=TH_EXTENT_RATIO,
            th_distance_av=TH_DISTANCE_AV,
        )
    agents_mask_origin = np.asarray(convenience.load(str(agents_mask_path)))

    # create chopped dataset
    zarr_scenes_chop(str(zarr_path), str(chopped_path), num_frames_to_copy=num_frames_to_copy)
    zarr_chopped = ChunkedDataset(str(chopped_path))
    zarr_chopped.open()

    # compute the chopped boolean mask, but also the original one limited to frames of interest for GT csv
    agents_mask_chop_bool = np.zeros(len(zarr_chopped.agents), dtype=np.bool)
    agents_mask_orig_bool = np.zeros(len(zarr_dt.agents), dtype=np.bool)

    for idx in range(len(zarr_dt.scenes)):
        scene = zarr_dt.scenes[idx]

        frame_original = zarr_dt.frames[scene["frame_index_interval"][0] + num_frames_to_copy - 1]
        slice_agents_original = get_agents_slice_from_frames(frame_original)
        frame_chopped = zarr_chopped.frames[zarr_chopped.scenes[idx]["frame_index_interval"][-1] - 1]
        slice_agents_chopped = get_agents_slice_from_frames(frame_chopped)

        mask = agents_mask_origin[slice_agents_original][:, 1] >= min_frame_future
        agents_mask_orig_bool[slice_agents_original] = mask.copy()
        agents_mask_chop_bool[slice_agents_chopped] = mask.copy()

    # store the mask and the GT csv of frames on interest
    np.savez(str(mask_chopped_path), agents_mask_chop_bool)
    export_zarr_to_csv(zarr_dt, str(gt_path), num_frames_gt, th_agent_prob, agents_mask=agents_mask_orig_bool)
    return str(dest_path)

## Carga y formato de dataset de evaluación
Dada la naturaleza del proyecto, el dataset de evaluación solo puede contener una porcion de los frames de cada escena (para este caso 100), para evitar que el modelo se adelante en la linea de tiempo incluida en el data set y utilice esos datos para la evaluación entregando resultados poco realistas. 

In [ ]:
# ===== GENERATE AND LOAD CHOPPED DATASET

# Número de frames que se quieren utilizar del data set de validación para la evaluación
# y evitar el problema de mirar "El futuro" y obtener un resultado impreciso sobre el desempeño del modelo
num_frames_to_chop = 100
# Carga de los datos de validación
eval_cfg = cfg["val_data_loader"]

# Creación y del nuevo CSV con unicamente los primeros 100 frames de las escenas del 
# data set de evaluación - función proporcionada en el l5kit pero modificada para funcionar en el kernel de kaggle
eval_base_path = xcreate_chopped_dataset(dm.require(eval_cfg["key"]), cfg["raster_params"]["filter_agents_threshold"], 
                              num_frames_to_chop, cfg["model_params"]["future_num_frames"], MIN_FUTURE_STEPS)

In [ ]:
# datos de evaluación
eval_zarr_path = str(Path(eval_base_path) / Path(dm.require(eval_cfg["key"])).name)
# máscara para los agentes
eval_mask_path = str(Path(eval_base_path) / "mask.npz")
# path del ground-truth para la evaluación
eval_gt_path = str(Path(eval_base_path) / "gt.csv")

# Descomprension de los datos de evaluación
eval_zarr = ChunkedDataset(eval_zarr_path).open()
eval_mask = np.load(eval_mask_path)["arr_0"]

# ===== INIT DATASET AND LOAD MASK
# Carga los datos de los agentes para la evaluación
eval_dataset = AgentDataset(cfg, eval_zarr, rasterizer, agents_mask=eval_mask)
# Carga los datos de evaluación
eval_dataloader = DataLoader(eval_dataset, shuffle=eval_cfg["shuffle"], batch_size=eval_cfg["batch_size"], 
                             num_workers=eval_cfg["num_workers"])
print(eval_dataset)

In [ ]:
# ==== EVAL LOOP
model.eval()
torch.set_grad_enabled(False)

# store information for evaluation
future_coords_offsets_pd = []
timestamps = []

agent_ids = []
progress_bar = tqdm(eval_dataloader)
# Loop de evaluación
for data in progress_bar:
    _, ouputs = forward(data, model, device, criterion)
    future_coords_offsets_pd.append(ouputs.cpu().numpy().copy())
    timestamps.append(data["timestamp"].numpy().copy())
    agent_ids.append(data["track_id"].numpy().copy())

## Se genera un archivo con las predicciones hechas por el modelo

In [ ]:
pred_path = f"{gettempdir()}/pred.csv"
# Escritura del CSV con las predicciones
write_pred_csv(pred_path,
               timestamps=np.concatenate(timestamps),
               track_ids=np.concatenate(agent_ids),
               coords=np.concatenate(future_coords_offsets_pd),
              )

### Cálculo de las metricas del desempeño del modelo: 


l5kit tiene predifinidas varias funciones para medir el desempeño del modelo, en este caso se implementara *time_displace* por su facilidad de interpretación en una primer iteración.

A la funcion de *compute_metrics_csv* se le entrega el GT, las predicciones, y una lista con las metricas que se quieren medir.

- time_displace = Desviación de la predicción en T timestamps en el sistemas de coordenadas globales 

In [ ]:
metrics = compute_metrics_csv(eval_gt_path, pred_path, [time_displace])
for metric_name, metric_mean in metrics.items():
    print(metric_name, metric_mean)

### Visualización de los resultados:
Para visualizar los resultados desde el punto de vista del vehículo autónomo del frame de interes (frame 100) se tienen que usar los datos de GT para poder visualizar las trayectorias futuras ya que los datos de evaluacion fueron cortados para el data set de evaluación. 

Para la visualización los datos de posicion de los agentes se guardan como coordenadas absolutas en el mundo con el origen en [37°25'45.6"N, 122°09'15.7"W] **Palo Alto California** y estan representadas como un set de coordenas 2D (X,Y) con una direccion respresentada como un yaw(*rotación sobre el eje Z*). Se utiliza este sistema dee coordenadas ya que el resot se utilizan para funciones de mas alto nivel con los data sets de ***Agent_dataset o Ego_dataset*** o tiene usos especificos para la identificación de diferentes agentes de tráfico dentro de la escena.

*Nota*: el dataset tiene varios sistemas de coordenadas disponibles: 
- Sistema de Coordenadas absolutas en el globo
- Sistema de Coordenadas satelitales 
- Sistema de Coordenadas de la imagen 
- Sistema de Coordenadas de los agentes
- Sistema de Coordenadas Semántico
La descripción de los diferentes sistemas de coordenadas se puedre encontrar en este [link](http://https://github.com/lyft/l5kit/blob/master/coords_systems.md#image-coordinate-system)

In [ ]:
model.eval()
torch.set_grad_enabled(False)

# build a dict to retrieve future trajectories from GT
gt_rows = {}
for row in read_gt_csv(eval_gt_path):
    gt_rows[row["track_id"] + row["timestamp"]] = row["coord"]

eval_ego_dataset = EgoDataset(cfg, eval_dataset.dataset, rasterizer)

for frame_number in range(99, len(eval_zarr.frames), 100):  # start from last frame of scene_0 and increase by 100
    agent_indices = eval_dataset.get_frame_indices(frame_number) 
    if not len(agent_indices):
        continue

    # get AV point-of-view frame
    data_ego = eval_ego_dataset[frame_number]
    im_ego = rasterizer.to_rgb(data_ego["image"].transpose(1, 2, 0))
    center = np.asarray(cfg["raster_params"]["ego_center"]) * cfg["raster_params"]["raster_size"]
    
    predicted_positions = []
    target_positions = []

    for v_index in agent_indices:
        data_agent = eval_dataset[v_index]

        out_net = model(torch.from_numpy(data_agent["image"]).unsqueeze(0).to(device))
        out_pos = out_net[0].reshape(-1, 2).detach().cpu().numpy()
        # store absolute world coordinates
        predicted_positions.append(transform_points(out_pos, data_agent["world_from_agent"]))
        # retrieve target positions from the GT and store as absolute coordinates
        track_id, timestamp = data_agent["track_id"], data_agent["timestamp"]
        target_positions.append(gt_rows[str(track_id) + str(timestamp)] + data_agent["centroid"][:2])


    # convert coordinates to AV point-of-view so we can draw them
    predicted_positions = transform_points(np.concatenate(predicted_positions), data_ego["raster_from_world"])
    target_positions = transform_points(np.concatenate(target_positions), data_ego["raster_from_world"])

    draw_trajectory(im_ego, predicted_positions, PREDICTED_POINTS_COLOR)
    draw_trajectory(im_ego, target_positions, TARGET_POINTS_COLOR)

    plt.imshow(im_ego)
    plt.show()

## Conclusiones:
En este primer acercamiento al problema basado en la documentacion ofrecida por lyft como solución base del problema. 
Los resultados del entrenamiento indican que las predicciones hechas por el sistema no son muy buenas ya que con el tiempo su desviación siempre crece.


En un futuro se buscara explorar e implementar nuevas métricas de para medir el desempeño del modelo e implementar un modelo con una salida multi-modal para resolver el problema.